In [1]:
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai import GenerationConfig
load_dotenv()
import json
import os

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genai.configure(api_key = os.getenv("GEMINI_KEY"))
config = GenerationConfig(temperature=0.9)
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction= 
                              '''You are an interior designer who specializes in furniture design and layout.
                              Your job is to be as accurate as possible in understanding floor plans.
                              Use any keywords present on the floor plan, such as "Bedroom", "Bathroom", "Living Room", etc.
                              to determine the layout of the apartment or house.
                              ''')


In [3]:
def analyze_floor_plan(file_path):
    try:
        floor_plan = genai.upload_file(file_path)
        response = model.generate_content(['''
                                        Analyze this floor plan.
                                        Store all the different rooms and spaces with their dimensions and any special features
                                        in a json format.
                                        Try to determine as much information about unspecified dimensions by using
                                        context clues of shapes like rectangles and squares to have a best estimate
                                        for at least one of the dimensions.
                                        Format the output in a json like this:
                                        {
                                            "Living Room": {
                                                "Dimensions": "20x15",
                                                "Special Features": "Fireplace, Large Windows"
                                            },
                                            "Kitchen": {
                                                "Dimensions": "15x10",
                                                "Special Features": "Island, Pantry"
                                            },
                                            "Bedroom1": {
                                                "Dimensions": "12x12",
                                                "Special Features": "Walk-in Closet"
                                            },
                                            "Bedroom2": {
                                                "Dimensions": "12x12",
                                                "Special Features": "Ensuite Bathroom"
                                            },
                                        }''', floor_plan],
                                        generation_config=config)
        genai.delete_file(floor_plan)
        return post_processing(response.text)
    except Exception as e:
        return "Check your file path or model configuration. Error: " + str(e)
def post_processing(output):
    line_separated = output.splitlines()
    starting_index = line_separated.index("```json")
    ending_index = line_separated.index("```", starting_index + 1)
    extracted_json = '\n'.join(line_separated[starting_index + 1:ending_index])
    return json.loads(extracted_json)
output = analyze_floor_plan("floor_plans/002.pdf")

In [4]:
print(json.dumps(output, indent = 4))

{
    "Living Room": {
        "Dimensions": "14'2\" x 12'3\"",
        "Special Features": []
    },
    "Kitchen": {
        "Dimensions": "10'8\" x 8'0\"",
        "Special Features": [
            "Sink",
            "Dishwasher",
            "Refrigerator"
        ]
    },
    "Bathroom": {
        "Dimensions": "6' x 7'",
        "Special Features": [
            "Tub"
        ]
    },
    "Foyer": {
        "Dimensions": "6' x 7'",
        "Special Features": []
    },
    "Closet": {
        "Dimensions": "3' x 4'",
        "Special Features": [
            "Washer/Dryer"
        ]
    },
    "Entry": {
        "Dimensions": "3' x 4'",
        "Special Features": []
    },
    "Reference": {
        "Dimensions": "3' x 4'",
        "Special Features": []
    }
}


In [139]:
import json
print(json.dumps(json.loads(output), indent = 4))

In [ ]:
for room in json.loads(output):
    print(f"Room: {room['room']}")
    print(f"Dimensions: {room['dimensions']}")
    print("\n")

In [179]:
import ikea_api

# Constants like country, language, base url
constants = ikea_api.Constants(country="us", language="en")
# Search API
search = ikea_api.Search(constants)
# Search endpoint with prepared data
endpoint = search.search("floor lamp")


In [180]:
results = ikea_api.run(endpoint)

In [181]:
print(json.dumps(results, indent = 4))

{
    "usergroup": "[]",
    "testActivationTriggers": {},
    "searchResultPage": {
        "searchPhrase": "floor lamp",
        "storeWindow": [],
        "storeCount": 0,
        "didYouMean": [],
        "relatedSearches": [
            {
                "count": 532,
                "text": "lamp",
                "metadata": "1498"
            },
            {
                "count": 156,
                "text": "table lamp",
                "metadata": "728"
            },
            {
                "count": 82,
                "text": "desk lamp",
                "metadata": "298"
            },
            {
                "count": 740,
                "text": "lighting",
                "metadata": "268"
            },
            {
                "count": 71,
                "text": "lamp shade",
                "metadata": "221"
            },
            {
                "count": 111,
                "text": "side table",
                "metadata": "202"
         